<a href="https://colab.research.google.com/github/SumaOladri/potential-waddle/blob/master/Chunking_Documents_Creating_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia-api

In [2]:
import numpy as np
import pandas as pd
import wikipediaapi

In [3]:
# Call the Wikipedia function through a user_agent and specify the language

string = "SemSearchDemo/1.0 (Akshay G;)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent= string, language= 'en')

In [5]:
wiki_wiki

In [6]:
# Extract the wikipedia page for the page - " The Shawshank Redemption"

page = wiki_wiki.page("The Shawshank Redemption")

In [7]:
page

The Shawshank Redemption (id: ??, ns: 0)

In [9]:
len(page.text)

51819

In [11]:
page.sections

[Section: Plot (1):
 In early 1947, Portland, Maine banker Andy Dufresne arrives at Shawshank State Prison to serve two consecutive life sentences for murdering his wife and her lover. He is befriended by Ellis "Red" Redding, a contraband smuggler serving a life sentence, who procures a rock hammer and a large poster of Rita Hayworth for Andy. Assigned to work in the prison laundry, Andy is frequently raped by "the Sisters" prison gang and their leader, Bogs Diamond. 
 In 1949, Andy overhears the captain of the guards, Byron Hadley, complaining about being taxed on an inheritance and offers to help him shelter the money legally. After an assault by the Sisters nearly kills Andy, Hadley beats and cripples Bogs, who is subsequently transferred to a minimum security hospital; Andy is not attacked again. Warden Samuel Norton meets Andy and reassigns him to the decrepit prison library to assist elderly inmate Brooks Hatlen, a front to use Andy's financial expertise to manage financial matte

FIXED SIZE CHUNKING

In [17]:
def split_text_chunks(text,chunk_size):
  chunks = []
  words = text.split()
  current_chunk_word_count = 0
  current_chunk = []
  for word in words:
    if current_chunk_word_count + len(word)+1<=chunk_size:
      current_chunk_word_count += len(word)+1
      current_chunk.append(word)
    else:
      chunks.append(' '.join(current_chunk))
      current_chunk = [word]
      current_chunk_word_count = len(word)
  if current_chunk:
    chunks.append(' '.join(current_chunk))
  return chunks

In [18]:
text = """In November 2014, the Academy of Motion Picture Arts and Sciences celebrated the film's 20th anniversary with a special one-night screening at the Samuel Goldwyn Theater in Beverly Hills, California. In 2015, the film was selected by the United States Library of Congress to be preserved in the National Film Registry for being "culturally, historically, or aesthetically significant". Darabont responded: "I can think of no greater honor than for The Shawshank Redemption to be considered part of our country's cinematic legacy." Variety said that the word "Shawshank" could be used to instantly convey images of a prison.
 The significant and enduring public appreciation for the film has often been difficult for critics to define. In an interview, Freeman said, "About everywhere you go, people say, 'The Shawshank Redemption—greatest movie I ever saw'" and that such praise "Just comes out of them". Robbins said, "I swear to God, all over the world—all over the world—wherever I go, there are people who say, 'That movie changed my life'". In a separate interview, Stephen King said, "If that isn't the best [adaptation of my works], it's one of the two or three best, and certainly, in moviegoers' minds, it's probably the best because it generally rates at the top of these surveys they have of movies. ... I never expected anything to happen with it." In a 2014 Variety article, Robbins claimed that South African politician Nelson Mandela told him about his love for the film, while it has been cited as a source of inspiration by several sportsmen including Jonny Wilkinson (UK), Agustín Pichot (Argentina), Al Charron (Canada), and Dan Lyle (USA), and Sarah Ferguson, the Duchess of York. Gunton said he had encountered fans in Morocco, Australia, South America, Germany, France, and Bora Bora. Director Steven Spielberg said that the film was "a chewing-gum movie—if you step on it, it sticks to your shoe". Speaking on the film's 25th anniversary, Darabont said that older generations were sharing it with younger generations, contributing to its longevity.
 It has been the number-one film on IMDb's user-generated Top 250 since 2008, when it surpassed The Godfather, having remained at or near the top since the late 1990s. In the United Kingdom, readers of Empire voted the film as the best of the 1990s, the greatest film of all time in 2006, and it placed number four on Empire's 2008 list of "The 500 Greatest Movies of All Time" and their 2017 list of "The 100 Greatest Movies". In March 2011, the film was voted by BBC Radio 1 and BBC Radio 1Xtra listeners as their favorite film of all time. It regularly appears on Empire's top 100 films, was named the greatest film to not win the Academy Award for Best Picture in a 2013 poll by Sky UK (it lost to Forrest Gump), and ranked as Britain's favorite film in a 2015 YouGov poll. When the British Film Institute analyzed the demographic breakdown of the YouGov poll, it noted that The Shawshank Redemption was not the top-ranked film in any group, but was the only film to appear in the top 15 of every age group, suggesting it is able to connect with every polled age group, unlike Pulp Fiction which fared better with younger voters, and Gone with the Wind (1939) with older voters.  """
chunk_size = 20
chunks = split_text_chunks(text,chunk_size)

In [19]:
print(chunks)

['In November 2014,', 'the Academy of', 'Motion Picture Arts', 'and Sciences', 'celebrated the', "film's 20th", 'anniversary with a', 'special one-night', 'screening at the', 'Samuel Goldwyn', 'Theater in Beverly', 'Hills, California.', 'In 2015, the film', 'was selected by the', 'United States', 'Library of Congress', 'to be preserved in', 'the National Film', 'Registry for being', '"culturally,', 'historically, or', 'aesthetically', 'significant".', 'Darabont responded:', '"I can think of no', 'greater honor than', 'for The Shawshank', 'Redemption to be', 'considered part of', "our country's", 'cinematic legacy."', 'Variety said that', 'the word "Shawshank"', 'could be used to', 'instantly convey', 'images of a prison.', 'The significant and', 'enduring public', 'appreciation for the', 'film has often been', 'difficult for', 'critics to define.', 'In an interview,', 'Freeman said, "About', 'everywhere you go,', "people say, 'The", 'Shawshank', 'Redemption—greatest', 'movie I ever saw

In [22]:
def process_page(page_title):
  page = wiki_wiki.page(page_title)
  if page.exists():
    text = page.text
    chunk_size = 1000
    chunks = split_text_chunks(text,chunk_size)
    data = {"Title":[],"Chunk_Text":[]}
    for idx,chunk in enumerate(chunks):
      data['Title'].append(page_title)
      data['Chunk_Text'].append(chunk)
    return pd.DataFrame(data)
  else:
    print(f"The page '{page_title}' does not exist on Wikipedia.")
    return None


In [23]:
page_titles = ["The Shawshank Redemption", "The Dark Knight", "Pulp Fiction", "The Godfather", "Goodfellas"]

all_dfs = []

for title in page_titles:
    df = process_page(title)
    if df is not None:
        all_dfs.append(df)


fixed_chunk_df = pd.concat(all_dfs, ignore_index=True)
fixed_chunk_df

,Title,Chunk_Text
0,The Shawshank Redemption,The Shawshank Redemption is a 1994 American pr...
1,The Shawshank Redemption,"Redemption, which started pre-production in Ja..."
2,The Shawshank Redemption,It went on to receive multiple award nominatio...
3,The Shawshank Redemption,"Registry, finding it ""culturally, historically..."
4,The Shawshank Redemption,front to use Andy's financial expertise to man...
...,...,...
272,Goodfellas,preservation in the National Film Registry. Ro...
273,Goodfellas,a very important movie to me and Goodfellas re...
274,Goodfellas,announced that the AMC Network had put a telev...
275,Goodfellas,Film Festival with a screening of its 25th-ann...


2.2 PARAGRAPH CHUNKING

In [25]:
# Defining some redundant sections

redundant_sections = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]

In [26]:
#Function to recursively extract paragraphs from sections
page = wiki_wiki.page("The Shawshank Redemption")

In [35]:
page.sections

[Section: Plot (1):
 In early 1947, Portland, Maine banker Andy Dufresne arrives at Shawshank State Prison to serve two consecutive life sentences for murdering his wife and her lover. He is befriended by Ellis "Red" Redding, a contraband smuggler serving a life sentence, who procures a rock hammer and a large poster of Rita Hayworth for Andy. Assigned to work in the prison laundry, Andy is frequently raped by "the Sisters" prison gang and their leader, Bogs Diamond. 
 In 1949, Andy overhears the captain of the guards, Byron Hadley, complaining about being taxed on an inheritance and offers to help him shelter the money legally. After an assault by the Sisters nearly kills Andy, Hadley beats and cripples Bogs, who is subsequently transferred to a minimum security hospital; Andy is not attacked again. Warden Samuel Norton meets Andy and reassigns him to the decrepit prison library to assist elderly inmate Brooks Hatlen, a front to use Andy's financial expertise to manage financial matte

In [32]:
for section in page.sections:
  print (section.title)

Plot
Cast
Analysis
Production
Release
Reception
Legacy
Notes
References
External links


In [36]:
section.sections

[]

In [33]:
page.summary

'The Shawshank Redemption is a 1994 American prison drama film written and directed by Frank Darabont, based on the 1982 Stephen King novella Rita Hayworth and Shawshank Redemption. The film tells the story of banker Andy Dufresne (Tim Robbins), who is sentenced to life in Shawshank State Penitentiary for the murders of his wife and her lover, despite his claims of innocence. Over the following two decades, he befriends a fellow prisoner, contraband smuggler Ellis "Red" Redding (Morgan Freeman), and becomes instrumental in a money laundering operation led by the prison warden Samuel Norton (Bob Gunton). William Sadler, Clancy Brown, Gil Bellows, and James Whitmore appear in supporting roles.\nDarabont purchased the film rights to King\'s story in 1987, but development did not begin until five years later, when he wrote the script over an eight-week period. Two weeks after submitting his script to Castle Rock Entertainment, Darabont secured a $25 million budget to produce The Shawshank 

In [48]:
def extract_paragraphs(section,acc_text):
  acc_text.extend(section.text.split('\n'))
  for subsection in section.sections:
     extract_paragraphs(subsection,acc_text)


In [44]:
def extract_wikepedia_paragraph(page_title):
  page = wiki_wiki.page(page_title)
  if page.exists():
    title = page.title
    paragraph = []
    if page.summary:
      paragraph.extend(page.summary.split('\n'))
    for section in page.sections:
      if section.title not in redundant_sections:
        extract_paragraphs(section,paragraph)
    return title,paragraph
  else:
    return None,None





In [68]:
page_titles = ["The Shawshank Redemption", "The Dark Knight", "Pulp Fiction", "The Godfather", "Goodfellas"]
# Initialize lists to store data
titles = []
paragraphs = []

# Extract data and store in lists
for title in page_titles:
    extracted_title, extracted_paragraphs = extract_wikepedia_paragraph(title)
    if extracted_title and extracted_paragraphs:
        titles.extend([extracted_title] * len(extracted_paragraphs))
        paragraphs.extend(extracted_paragraphs)

# Create a DataFrame
data = {'Title': titles, 'Chunk_Text': paragraphs}
para_chunk_df = pd.DataFrame(data)

# Display the DataFrame
para_chunk_df = para_chunk_df[para_chunk_df['Chunk_Text'] != '']  # Remove empty paragraphs
para_chunk_df = para_chunk_df.reset_index(drop=True)   # Reset index
para_chunk_df

,Title,Chunk_Text
0,The Shawshank Redemption,The Shawshank Redemption is a 1994 American pr...
1,The Shawshank Redemption,Darabont purchased the film rights to King's s...
2,The Shawshank Redemption,While The Shawshank Redemption received critic...
3,The Shawshank Redemption,"Over 320,000 VHS rental copies were shipped th..."
4,The Shawshank Redemption,"In early 1947, Portland, Maine banker Andy Duf..."
...,...,...
427,Goodfellas,AFI's 100 Years... 100 Movies - #94
428,Goodfellas,AFI's 100 Years... 100 Movies (10th Anniversar...
429,Goodfellas,AFI's 10 Top 10 - #2 Gangster film
430,Goodfellas,AFI's 100 Years... 100 Heroes and Villains - T...


CHUNKING BY SECTIONS

In [57]:
def extract_wikipedia_sections(page_title):
  page = wiki_wiki.page(page_title)
  if page.exists():
    title = page.title
    sections = []
    if page.summary:
      sections.append(("Introduction",page.summary))
    def recursive_extraction(section):
      section_text = section.text
      if section_text:
        sections.append((section.title,section_text))
      for subsection in section.sections:
        recursive_extraction(subsection)
    for section in page.sections:
      if section.title not in redundant_sections:
          recursive_extraction(section)
    return title, sections

  else:
      return None, None






In [67]:
# List of Wikipedia page titles
page_titles = ["The Shawshank Redemption", "The Dark Knight", "Pulp Fiction", "The Godfather", "Goodfellas"]

# Initialize lists to store data
titles = []
section_titles = []
section_texts = []

# Extract data and store in lists
for title in page_titles:
    extracted_title, extracted_sections = extract_wikipedia_sections(title)
    if extracted_title and extracted_sections:
        for section_title, section_text in extracted_sections:
            titles.append(extracted_title)
            section_titles.append(section_title)
            section_texts.append(section_text)

# Create a DataFrame
data = {'Title': titles, 'Section_Title': section_titles, 'Chunk_Text': section_texts}
section_chunk_df = pd.DataFrame(data)

# Display the DataFrame
section_chunk_df = section_chunk_df[section_chunk_df['Chunk_Text'] != '']  # Remove empty sections
section_chunk_df = section_chunk_df.reset_index(drop=True)  # Reset index
section_chunk_df

,Title,Section_Title,Chunk_Text
0,The Shawshank Redemption,Introduction,The Shawshank Redemption is a 1994 American pr...
1,The Shawshank Redemption,Plot,"In early 1947, Portland, Maine banker Andy Duf..."
2,The Shawshank Redemption,Cast,Tim Robbins as Andy Dufresne: A banker sentenc...
3,The Shawshank Redemption,Analysis,The film has been interpreted as being grounde...
4,The Shawshank Redemption,Development,Darabont first collaborated with author Stephe...
...,...,...,...
93,Goodfellas,Box office,"Goodfellas grossed $6.3 million from 1,070 the..."
94,Goodfellas,Critical response,According to review aggregator Rotten Tomatoes...
95,Goodfellas,Lists,The film was ranked the best of 1990 by Roger ...
96,Goodfellas,Accolades,It became one of the eight films to win Best P...


In [59]:
#Create embeddings

In [22]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [26]:
from sentence_transformers import SentenceTransformer, util

In [24]:
model_name = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
def generate_embeddings(text):
  embeddings = embedder.encode(text,convert_to_tensor=True)
  return embeddings

In [19]:
def generate_embeddings_on_df(df):
  df['Embeddings'] = df['Chunk_Text'].apply(lambda x: generate_embeddings([x])[0])

In [76]:
#generate_embeddings_on_df(fixed_chunk_df)
#generate_embeddings_on_df(para_chunk_df)
#generate_embeddings_on_df(section_chunk_df)

In [74]:
fixed_chunk_df

,Title,Chunk_Text,Embeddings
0,The Shawshank Redemption,The Shawshank Redemption is a 1994 American pr...,"[tensor(-0.0475), tensor(-0.0168), tensor(-0.0..."
1,The Shawshank Redemption,"Redemption, which started pre-production in Ja...","[tensor(-0.0266), tensor(-0.0215), tensor(-0.0..."
2,The Shawshank Redemption,It went on to receive multiple award nominatio...,"[tensor(0.0273), tensor(-0.0148), tensor(-0.00..."
3,The Shawshank Redemption,"Registry, finding it ""culturally, historically...","[tensor(-0.0202), tensor(-0.0067), tensor(-0.0..."
4,The Shawshank Redemption,front to use Andy's financial expertise to man...,"[tensor(-0.0097), tensor(-0.0166), tensor(-0.0..."
...,...,...,...
272,Goodfellas,preservation in the National Film Registry. Ro...,"[tensor(-0.0159), tensor(-0.0458), tensor(-0.0..."
273,Goodfellas,a very important movie to me and Goodfellas re...,"[tensor(-0.0505), tensor(-0.0296), tensor(-0.0..."
274,Goodfellas,announced that the AMC Network had put a telev...,"[tensor(-0.1081), tensor(0.0258), tensor(-0.10..."
275,Goodfellas,Film Festival with a screening of its 25th-ann...,"[tensor(-0.0927), tensor(-0.0510), tensor(-0.0..."


In [77]:
section_chunk_df

,Title,Section_Title,Chunk_Text,Embeddings
0,The Shawshank Redemption,Introduction,The Shawshank Redemption is a 1994 American pr...,"[tensor(-0.0452), tensor(-0.0287), tensor(-0.0..."
1,The Shawshank Redemption,Plot,"In early 1947, Portland, Maine banker Andy Duf...","[tensor(0.0084), tensor(0.0089), tensor(-0.047..."
2,The Shawshank Redemption,Cast,Tim Robbins as Andy Dufresne: A banker sentenc...,"[tensor(-0.0502), tensor(-0.0536), tensor(-0.1..."
3,The Shawshank Redemption,Analysis,The film has been interpreted as being grounde...,"[tensor(-0.0556), tensor(0.0511), tensor(-0.08..."
4,The Shawshank Redemption,Development,Darabont first collaborated with author Stephe...,"[tensor(-0.0338), tensor(-0.0343), tensor(-0.0..."
...,...,...,...,...
93,Goodfellas,Box office,"Goodfellas grossed $6.3 million from 1,070 the...","[tensor(0.0283), tensor(-0.0623), tensor(-0.01..."
94,Goodfellas,Critical response,According to review aggregator Rotten Tomatoes...,"[tensor(-0.0106), tensor(-0.0307), tensor(-0.1..."
95,Goodfellas,Lists,The film was ranked the best of 1990 by Roger ...,"[tensor(-0.0047), tensor(-0.0954), tensor(-0.0..."
96,Goodfellas,Accolades,It became one of the eight films to win Best P...,"[tensor(-0.0066), tensor(-0.0777), tensor(-0.0..."


In [81]:
len(para_chunk_df['Embeddings'][0])


384

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
output_path = '/content/drive/My Drive/Helpmate_AI_Project/'

In [84]:
# Save the dataframe with embeddings
#fixed_chunk_df.to_csv(output_path+"fixed_chunk_embeddings.csv", index=False)
#para_chunk_df.to_csv(output_path+"para_chunk_embeddings.csv", index=False)
#section_chunk_df.to_csv(output_path+"section_chunk_embeddings.csv", index=False)

In [9]:
import pandas as pd

In [15]:
fixed_chunk_df = pd.read_csv(output_path+"fixed_chunk_embeddings.csv")
para_chunk_df = pd.read_csv(output_path+"para_chunk_embeddings.csv")
section_chunk_df = pd.read_csv(output_path+"section_chunk_embeddings.csv")

In [16]:
# Read user input query
user_query = "Who are the cast in Goodfellas"

In [27]:
def calculate_similarity(embedding1,embedding2):
  cosine_score = util.pytorch_cos_sim(embedding1,embedding2)
  similarity = cosine_score.item()
  return similarity

In [6]:
def semantic_search(user_query, df, embedder):
  user_embedding = generate_embeddings(user_query)

  df['similarity'] = df['Embeddings'].apply(lambda x : calculate_similarity(x,user_embedding))

  df = df.sort_values(by = 'similarity',ascending=False ).reset_index(drop=True)
  #df.drop('Embeddings',axis=1,inplace=True)
  final_df = df.head(3)

  return final_df

In [28]:
# Perform semantic search on each DataFrame
fixed_chunk_results = semantic_search(user_query, fixed_chunk_df, embedder)
para_chunk_results = semantic_search(user_query, para_chunk_df, embedder)
section_chunk_results = semantic_search(user_query, section_chunk_df, embedder)

TypeError: new(): invalid data type 'str'

In [ ]:
fixed_chunk_results

In [ ]:
para_chunk_results

In [ ]:
section_chunk_results